In [70]:
import os
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import nltk

In [71]:
#функция отделяет авторскую речь от диалогов в книгах каждого автора 
#для каждого автора создается два файла:
#1. с диалогами из всех его книг dialog_[имя автора].txt
#2. с авторской речью из всех его книг author_[имя автора].txt

def separate_dialogs_from_author_speech(author):
    
    files = os.listdir(author)
    
    dialogs = []
    authors_speech = []
    
    for file in files:
        strfile = str(file)
        if strfile.endswith('.txt'):
            path = author + '/' + file
            with open(path, 'r', encoding = 'utf-8') as t:
                text = [line.rstrip('\n') for line in t]
                for string in text:
                    if string.startswith('–'):
                        #dashcounter = string.count('–')
                        #if dashcounter > 1:
                        dialogs.append(string)
                        dialogs.append('\n')
                    else:
                        if string != '':
                            authors_speech.append(string)
                            authors_speech.append('\n')
                    
            dialog_file = 'dialog_' + author
            author_file = 'author_' + author
        
            with open (dialog_file, 'w', encoding = 'utf-8') as file:
                for i in dialogs:
                    file.write(i)
            with open (author_file, 'w', encoding = 'utf-8') as file:
                for i in authors_speech:
                    file.write(i)

In [3]:
#функция чистит каждый файл в папке автора от аннотаций, имен авторов, обозначений глав

def cleaning(author):
    
    files = os.listdir(author)
    
    for file in files:
        strfile = str(file)
        if strfile.endswith('.txt'):
            path = author + '/' + file
            with open(path, 'r', encoding = 'utf-8') as t:
                text = t.read()
                thetext = re.sub('Annotation\n*.*', '', text)
                thetext = re.sub('© .*', '', thetext)
                thetext = re.sub('Глава \d*', '', thetext)
                thetext = re.sub('Анна и Сергей Литвиновы', '', thetext)
                thetext = re.sub('Татьяна Полякова', '', thetext)
                thetext = re.sub('Дарья Донцова', '', thetext)
                thetext = re.sub('Евгения Горская', '', thetext)
                thetext = re.sub('Александра Маринина', '', thetext)
                thetext = re.sub('Татьяна УСТИНОВА', '', thetext)
            with open(path, 'w+', encoding = 'utf-8') as t:
                newtext = t.write(thetext)

In [4]:
#функция считает слова во всех книгах каждого автора 

def word_counter(author):

    files = os.listdir(author)
    words = 0
    
    for file in files:
        strfile = str(file)
        if strfile.endswith('.txt'):
            path = author + '/' + file
            with open(path, 'r', encoding = 'utf-8') as t:
                text = t.read()
                one_file_words = len({x for x in re.findall(r'\w+', text)})
            words = words + one_file_words
    
    print(author, ': ', words)

In [5]:
for folder in os.listdir():
    strfolder = str(folder)
    if not strfolder.startswith('.') and not strfolder.endswith('.ipynb'):
        cleaning(folder)

In [24]:
for folder in os.listdir():
    strfolder = str(folder)
    if not strfolder.startswith('.') and not strfolder.endswith('.ipynb'):
        separate_dialogs_from_author_speech(folder)

## Количество слов у каждого автора:

In [5]:
for folder in os.listdir():
    if os.path.isdir(folder) and not str(folder).startswith('.'):
        word_counter(folder)

Ustinova :  739937
corpus :  0
Marinina :  1005699
Litvinovy :  1006743
Dontsova :  0
Polyakova :  1007637


## Количественные показатели

In [38]:
string = '– Спасибо, друг, – с чувством сказал убийца, поспешно откручивая колпачок с литровой ледяной бутылки нарзана. – Выручил ты меня. А что это ты с шариком делаешь?'
speech = re.search('(–\w*\s*)(–.*–*)(.*)', string)
print(speech.group(1))

– Спасибо, друг, – с чувством сказал убийца, поспешно откручивая колпачок с литровой ледяной бутылки нарзана. 


In [59]:
def count_everything(file):
    with open(file, 'r', encoding = 'utf-8') as t:
        text = t.read()
        split_regex = re.compile(r'[.|!|?|…|:|,|\n]')
        lenghts = []
        exclamations = 0
        questions = 0
        for string in text.split('\n'):
            string = string.replace('–', '')
            replicas = filter(lambda t: t, [t.strip() for t in split_regex.split(string)])
            for replica in replicas:
                len_replica = len(replica.split(' '))
                lenghts.append(len_replica)
            r = [t for t in split_regex.split(string)]
            for replica in r:
                if replica.endswith('!'):
                    exclamations + 1
                if replica.endswith('?'):
                    questions + 1
        count_words = len([x for x in re.findall(r'\w+', text)])
        every_punct = []
        punct = ['.', '!', '?', ':', ',']
        for i in text:
            for p in punct:
                if i in p:
                    every_punct.append(i)
        count_punct = len(every_punct)
        words_vs_punct = round(count_words / count_punct, 2)
        main_length = round(sum(lenghts) / len(lenghts), 2)
        #print(file, '\n', 'количество слов:', count_words,  '\n', 'ср. длина фразы:', main_length,  '\n', 
              #'соотношение слов и знаков препинания:', words_vs_punct, '\n', 'доля вопросительных реплик:', questions, '\n',
              #'доля восклицательных реплик:', exclamations, '\n',
              #'\n\n',)
        result = [file, count_words, main_length, words_vs_punct, questions, exclamations]
        return result

In [61]:
files = os.listdir('./corpus/')
for_df = []
for file in files: 
    path = './corpus/' + file
    if not file.startswith('.'):
        result = count_everything(path)
    for_df.append(result)
    print(result)
df = pd.DataFrame(for_df, 
columns=['название файла', 'количество слов','ср. длина фразы', 'соотношение слов и знаков препинания', 'доля вопросительных реплик', 'доля восклицательных реплик'])

['./corpus/dialog_Litvinovy', 1066781, 3.38, 3.4, 0, 0]
['./corpus/dialog_Litvinovy', 1066781, 3.38, 3.4, 0, 0]
['./corpus/dialog_Marinina', 1622804, 3.93, 4.01, 0, 0]
['./corpus/dialog_Ustinova', 1090179, 3.41, 3.37, 0, 0]
['./corpus/author_Polyakova', 3285514, 4.27, 4.21, 0, 0]
['./corpus/dialog_Polyakova', 1584808, 3.53, 3.67, 0, 0]
['./corpus/author_Ustinova', 2247227, 4.52, 4.36, 0, 0]
['./corpus/author_Marinina', 3220125, 4.85, 4.86, 0, 0]
['./corpus/author_Litvinovy', 2533700, 4.5, 4.52, 0, 0]


In [62]:
df

,название файла,количество слов,ср. длина фразы,соотношение слов и знаков препинания,доля вопросительных реплик,доля восклицательных реплик
0,./corpus/dialog_Litvinovy,1066781,3.38,3.40,0,0
1,./corpus/dialog_Litvinovy,1066781,3.38,3.40,0,0
2,./corpus/dialog_Marinina,1622804,3.93,4.01,0,0
3,./corpus/dialog_Ustinova,1090179,3.41,3.37,0,0
4,./corpus/author_Polyakova,3285514,4.27,4.21,0,0
5,./corpus/dialog_Polyakova,1584808,3.53,3.67,0,0
6,./corpus/author_Ustinova,2247227,4.52,4.36,0,0
7,./corpus/author_Marinina,3220125,4.85,4.86,0,0
8,./corpus/author_Litvinovy,2533700,4.50,4.52,0,0


In [63]:
df.to_csv('first_results.csv')